#Kuler: Cleaning the data

After scraping the JSON response, here I clean and parse the data using pandas. First, I import JSON response as a df format using read_json in pandas. Then, I drop some data columns that I will not use for now for simplicity.


Here's the key values from the raw data:
<p>
- author: author Id and name
- comment: number of comments
- createAt: date created
- tags: text description of the theme
- description
- harmony: information about creating process of the theme (e.g., rule)
- href: url
- originalTheme: if the theme is created based on an already-existing theme, its value is the id of that parent theme
- access
- like: number of likes
- view: number of views
- rating: review count and average rating
- iccProfiles: ?
- swatches: color information in RGB, hex, and colorIndex (?)
- name: name of the theme
- id: theme id
- editedAt: latest date edited 
<p>

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import scipy as sp

# plotting
import matplotlib.pyplot as plt
import matplotlib.cm as pltcm
%matplotlib inline

##I. Import the json response using read_json

In [2]:
# read_json: convert json into dataframe
df = pd.read_json('kuler_data_new.txt')
df.head()

,access,author,comment,createdAt,description,editedAt,harmony,href,iccProfile,id,like,name,originalTheme,rating,swatches,tags,view
0,{u'visibility': u'public'},"{u'id': u'17923', u'name': u'ps'}",{u'count': 339},2007-01-18T08:51:12+00:00,,2014-12-16T05:24:50+00:00,"{u'baseSwatchIndex': 0, u'sourceURL': u'', u'm...",http://adobe.ly/1BOf1GL,,15325,{u'count': 13707},sandy stone beach ocean diver,0,"{u'count': 1755, u'overall': 3.9}","[{u'hex': u'E6E2AF', u'colorIndex': 598, u'val...","[{u'value': u'beach'}, {u'value': u'diver'}, {...",{u'count': 39612}
1,{u'visibility': u'public'},"{u'id': u'17772', u'name': u'matthepworth'}",{u'count': 186},2007-03-01T00:42:36+00:00,,2007-03-01T00:51:15+00:00,"{u'baseSwatchIndex': 0, u'sourceURL': u'', u'm...",https://kuler.adobe.com/Firenze-color-theme-24...,,24198,{u'count': 10737},Firenze,0,"{u'count': 1441, u'overall': 3.89}","[{u'hex': u'468966', u'colorIndex': 296, u'val...","[{u'value': u'bellagio'}, {u'value': u'boot'},...",{u'count': 28750}
2,{u'visibility': u'public'},"{u'id': u'17772', u'name': u'matthepworth'}",{u'count': 54},2007-02-22T00:06:18+00:00,,2007-02-22T00:06:18+00:00,"{u'baseSwatchIndex': 0, u'sourceURL': u'', u'm...",https://kuler.adobe.com/Neutral-Blue-color-the...,,22361,{u'count': 8993},Neutral Blue,0,"{u'count': 556, u'overall': 3.78}","[{u'hex': u'FCFFF5', u'colorIndex': 1558, u'va...","[{u'value': u'adult'}, {u'value': u'azure'}, {...",{u'count': 18051}
3,{u'visibility': u'public'},"{u'id': u'243670', u'name': u'rockstarflu'}",{u'count': 18},2012-02-14T16:36:28+00:00,,2012-02-14T16:36:28+00:00,"{u'baseSwatchIndex': 1, u'sourceURL': u'', u'm...",https://kuler.adobe.com/Phaedra-color-theme-17...,,1764754,{u'count': 7108},Phaedra,0,"{u'count': 149, u'overall': 4.06}","[{u'hex': u'FF6138', u'colorIndex': 1884, u'va...",[],{u'count': 10185}
4,{u'visibility': u'public'},"{u'id': u'422481', u'name': u'dezi9er'}",{u'count': 50},2011-09-03T16:14:04+00:00,,2011-09-03T16:14:04+00:00,"{u'baseSwatchIndex': 2, u'sourceURL': u'', u'm...",https://kuler.adobe.com/Honey-Pot-color-theme-...,,1490158,{u'count': 6352},Honey Pot,0,"{u'count': 261, u'overall': 4.09}","[{u'hex': u'105B63', u'colorIndex': 781, u'val...","[{u'value': u'bee'}, {u'value': u'bottle'}, {u...",{u'count': 9708}


##II. Clean data

In [3]:
# drop the columns that I am not going to use for now
df = df.drop('access', 1)
df = df.drop('description', 1)
df = df.drop('harmony', 1)
df = df.drop('iccProfile', 1)
df = df.drop('originalTheme', 1)
df = df.drop('editedAt', 1)
df = df.drop('href', 1)

In [4]:
# tip: use apply and infer_dtype for checking dtypes
df.apply(lambda x: pd.lib.infer_dtype(x.values))

author         mixed
comment        mixed
createdAt    unicode
id           integer
like           mixed
name         unicode
rating         mixed
swatches       mixed
tags           mixed
view           mixed
dtype: object

##III. Explore data formats

###1. Time

In [5]:
# import datetime
from datetime import datetime
df_time_example = df.createdAt[0]
df_time_example = df_time_example[:-6] # I left out +00:00 part
print df_time_example
datetime.strptime(df_time_example, '%Y-%m-%dT%H:%M:%S')

2007-01-18T08:51:12


datetime.datetime(2007, 1, 18, 8, 51, 12)

###2. Color

In [10]:
# df.swatches[i]: list of dicts
df.swatches[0]

[{u'colorIndex': 598,
  u'hex': u'E6E2AF',
  u'mode': u'rgb',
  u'name': u'',
  u'values': [0.901961, 0.8862749999999999, 0.686275]},
 {u'colorIndex': 1630,
  u'hex': u'A7A37E',
  u'mode': u'rgb',
  u'name': u'',
  u'values': [0.654902, 0.639216, 0.494118]},
 {u'colorIndex': 594,
  u'hex': u'EFECCA',
  u'mode': u'rgb',
  u'name': u'',
  u'values': [0.937255, 0.92549, 0.792157]},
 {u'colorIndex': 854,
  u'hex': u'046380',
  u'mode': u'rgb',
  u'name': u'',
  u'values': [0.0156863, 0.388235, 0.501961]},
 {u'colorIndex': 1544,
  u'hex': u'002F2F',
  u'mode': u'rgb',
  u'name': u'',
  u'values': [0, 0.18431399999999998, 0.18431399999999998]}]

In [12]:
df.swatches[0][0]

{u'colorIndex': 598,
 u'hex': u'E6E2AF',
 u'mode': u'rgb',
 u'name': u'',
 u'values': [0.901961, 0.8862749999999999, 0.686275]}

In [13]:
# We only need hexcode
str(df.swatches[0][0]["hex"])

'E6E2AF'

##IV. Extract data

###1. Decide which key-value to extract
Column name - key:
    - author: id
    - comment: count
    - like: count
    - rating: count, overall
    - view: count

In [14]:
def func(x,keyname):
    """Extract values using keys"""
    k = x[keyname] # here I read key values 
    return pd.Series(k)

###2. Extract values from dictionaries (general)

In [15]:
# Use pandas apply and lambda 
df['AuthorID']=df['author'].apply((lambda x: func(x,'id')))
df['Comments']=df['comment'].apply((lambda x: func(x,'count')))
df['Likes']=df['like'].apply((lambda x: func(x,'count')))
df['ReviewCounts']=df['rating'].apply((lambda x: func(x,'count')))
df['ReviewAvr']=df['rating'].apply((lambda x: func(x,'overall')))
df['Views']=df['view'].apply((lambda x: func(x,'count')))

In [16]:
# Drop the old columns
df = df.drop('author', 1)
df = df.drop('comment', 1)
df = df.drop('like', 1)
df = df.drop('rating', 1)
df = df.drop('view', 1)

###3. Extract the time data

In [17]:
def convert_time(x):
    """Read unicode time data and convert it to %Y-%m-%dT%H:%M:%S format. The last 6 strings will be truncated."""
    k = x[:-6]
    k = datetime.strptime(k, '%Y-%m-%dT%H:%M:%S')
    return pd.Series(k)

In [18]:
# Fix the time data format
df['Time']=df['createdAt'].apply((lambda x: convert_time(x)))
# Drop the old column
df = df.drop('createdAt', 1)

###4. Extract the color data
We build a separate column for each color (in order). The "swatches" column has this following structure: 
- df.swatches[i][j]["hex"]: i is the row number, and j is the (j+1)th color in the theme.

In [19]:
def func_extract_color(x,colorIdx):
    """Read a value from a df (x) using colorIdx, and extracts hexcode"""
    k = x[colorIdx]["hex"] # here I read key values 
    return pd.Series(k)

In [20]:
# Create new columns for each color
df['C1']=df['swatches'].apply((lambda x: func_extract_color(x,0)))
df['C2']=df['swatches'].apply((lambda x: func_extract_color(x,1)))
df['C3']=df['swatches'].apply((lambda x: func_extract_color(x,2)))
df['C4']=df['swatches'].apply((lambda x: func_extract_color(x,3)))
df['C5']=df['swatches'].apply((lambda x: func_extract_color(x,4)))
# Drop the old column
df = df.drop('swatches', 1)

###5. Rearrange columns

In [21]:
df = df[['id','name','C1','C2','C3','C4','C5','Likes','Time','AuthorID','ReviewCounts','ReviewAvr','Views','Comments','tags']]
df.head()

,id,name,C1,C2,C3,C4,C5,Likes,Time,AuthorID,ReviewCounts,ReviewAvr,Views,Comments,tags
0,15325,sandy stone beach ocean diver,E6E2AF,A7A37E,EFECCA,046380,002F2F,13707,2007-01-18 08:51:12,17923,1755,3.90,39612,339,"[{u'value': u'beach'}, {u'value': u'diver'}, {..."
1,24198,Firenze,468966,FFF0A5,FFB03B,B64926,8E2800,10737,2007-03-01 00:42:36,17772,1441,3.89,28750,186,"[{u'value': u'bellagio'}, {u'value': u'boot'},..."
2,22361,Neutral Blue,FCFFF5,D1DBBD,91AA9D,3E606F,193441,8993,2007-02-22 00:06:18,17772,556,3.78,18051,54,"[{u'value': u'adult'}, {u'value': u'azure'}, {..."
3,1764754,Phaedra,FF6138,FFFF9D,BEEB9F,79BD8F,00A388,7108,2012-02-14 16:36:28,243670,149,4.06,10185,18,[]
4,1490158,Honey Pot,105B63,FFFAD5,FFD34E,DB9E36,BD4932,6352,2011-09-03 16:14:04,422481,261,4.09,9708,50,"[{u'value': u'bee'}, {u'value': u'bottle'}, {u..."


##V. Save the data
We can pi